---
Docker Registry by Helm

In [ ]:
wget https://get.helm.sh/helm-v3.14.4-linux-amd64.tar.gz

--2024-04-11 16:19:34--  https://get.helm.sh/helm-v3.14.4-linux-amd64.tar.gz
Resolving get.helm.sh (get.helm.sh)... 152.199.39.108, 2606:2800:247:1cb7:261b:1f9c:2074:3c
Connecting to get.helm.sh (get.helm.sh)|152.199.39.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16162235 (15M) [application/x-tar]
Saving to: ‘helm-v3.14.4-linux-amd64.tar.gz’

helm-v3.14.4-linux- 100%[===================>]  15.41M  2.74MB/s    in 5.1s    

2024-04-11 16:19:40 (3.01 MB/s) - ‘helm-v3.14.4-linux-amd64.tar.gz’ saved [16162235/16162235]



In [ ]:
tar -xvzf helm-v3.14.4-linux-amd64.tar.gz

linux-amd64/
linux-amd64/README.md
linux-amd64/LICENSE
linux-amd64/helm


In [ ]:
sudo mv linux-amd64/helm /usr/local/bin/helm

In [ ]:
rm -rf linux-amd64

---
deploy

In [ ]:
helm repo add stable https://charts.helm.sh/stable
helm repo update

In [ ]:
helm repo list

NAME                	URL                                    
stable              	https://charts.helm.sh/stable          
kubernetes-dashboard	https://kubernetes.github.io/dashboard/


In [ ]:
helm upgrade --install docker-registry docker-registry --repo https://helm.twun.io --namespace docker-registry --create-namespace

NAME: docker-registry
LAST DEPLOYED: Sat Apr 13 21:23:45 2024
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
1. Get the application URL by running these commands:
  export POD_NAME=$(kubectl get pods --namespace default -l "app=docker-registry,release=docker-registry" -o jsonpath="{.items[0].metadata.name}")
  echo "Visit http://127.0.0.1:8080 to use your application"
  kubectl -n default port-forward $POD_NAME 8080:5000


In [ ]:
helm list -A

In [ ]:
export POD_NAME=$(
  kubectl get pods --namespace docker-registry -l "app=docker-registry,release=docker-registry" -o jsonpath="{.items[0].metadata.name}"
)

In [ ]:
kubectl get pod $POD_NAME -n docker-registry -o wide

In [ ]:
kubectl logs $POD_NAME --namespace docker-registry

In [ ]:
kubectl describe pod $POD_NAME --namespace docker-registry

In [ ]:
helm uninstall docker-registry -n docker-registry

release "docker-registry" uninstalled


In [1]:
kubectl delete namespace docker-registry

namespace "docker-registry" deleted


---
deploy

https://kubernetes.github.io/ingress-nginx/examples/docker-registry/

In [2]:
kubectl apply -f https://raw.githubusercontent.com/kubernetes/ingress-nginx/main/docs/examples/docker-registry/deployment.yaml

namespace/docker-registry created
deployment.apps/docker-registry created
service/docker-registry created


In [ ]:
kubectl get pods -A

NAMESPACE              NAME                                                   READY   STATUS      RESTARTS   AGE
docker-registry        docker-registry-68f8d8dc67-qkp57                       1/1     Running     0          26s
ingress-nginx          ingress-nginx-admission-create-qskjp                   0/1     Completed   0          57m
ingress-nginx          ingress-nginx-admission-patch-dbg69                    0/1     Completed   1          57m
ingress-nginx          ingress-nginx-controller-7dcdbcff84-jrccz              1/1     Running     0          57m
kube-flannel           kube-flannel-ds-lcndk                                  1/1     Running     0          7h22m
kube-flannel           kube-flannel-ds-lpr4m                                  1/1     Running     0          7h27m
kube-system            coredns-76f75df574-7sbbl                               1/1     Running     0          7h17m
kube-system            coredns-76f75df574-zmsv2                               1/1     Runn

In [ ]:
export POD_NAME=$(
  kubectl get pods -A -l 'app=docker-registry' -o jsonpath='{.items[0].metadata.name}'
)

In [ ]:
kubectl get pod $POD_NAME -n docker-registry -o wide

NAME                               READY   STATUS    RESTARTS   AGE    IP            NODE         NOMINATED NODE   READINESS GATES
docker-registry-68f8d8dc67-pcbbr   1/1     Running   0          100m   10.244.1.25   msd-npc112   <none>           <none>


In [ ]:
echo 'Visit http://127.0.0.1:5200 to use your application'

In [ ]:
kubectl port-forward $POD_NAME -n docker-registry 5200:5000

In [3]:
kubectl get service docker-registry -n docker-registry

NAME              TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
docker-registry   ClusterIP   10.110.50.247   <none>        5000/TCP   71s


In [ ]:
kubectl port-forward service/docker-registry -n docker-registry 5200:5000

Forwarding from 127.0.0.1:5200 -> 5000
Forwarding from [::1]:5200 -> 5000


---
service

In [ ]:
kubectl get service docker-registry -n default -o yaml > docker-registry-service.yaml

In [ ]:
cat << EOF > docker-registry-service.yaml
apiVersion: v1
kind: Service
metadata:
  annotations:
    meta.helm.sh/release-name: docker-registry
    meta.helm.sh/release-namespace: default
  name: docker-registry
  namespace: docker-registry
spec:
  type: ClusterIP
  ports:
  - port: 5000
    targetPort: 5000
EOF

In [ ]:
kubectl apply -f docker-registry-service.yaml

In [ ]:
kubectl get service docker-registry -n docker-registry

---
service

In [ ]:
cat << EOF > docker-registry-service.yaml
apiVersion: v1
kind: Service
metadata:
  annotations:
    meta.helm.sh/release-name: docker-registry
    meta.helm.sh/release-namespace: default
  name: docker-registry
  namespace: docker-registry
spec:
  type: NodePort
  ports:
  - port: 5000
    targetPort: 5000
    nodePort: 32500
EOF

In [ ]:
kubectl apply -f docker-registry-service.yaml

service/docker-registry configured


In [ ]:
kubectl get service docker-registry -n docker-registry

NAME              TYPE       CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE
docker-registry   NodePort   10.98.106.37   <none>        5000:32500/TCP   36m


---
service

In [ ]:
cat << EOF > docker-registry-service.yaml
apiVersion: v1
kind: Service
metadata:
  annotations:
    meta.helm.sh/release-name: docker-registry
    meta.helm.sh/release-namespace: default
  name: docker-registry
  namespace: docker-registry
spec:
  externalIPs:
    - 172.28.242.11
  ports:
  - port: 5000
    targetPort: 5000
EOF

In [ ]:
kubectl apply -f docker-registry-service.yaml

service/docker-registry configured


In [ ]:
kubectl get service docker-registry -n docker-registry

NAME              TYPE        CLUSTER-IP     EXTERNAL-IP     PORT(S)    AGE
docker-registry   ClusterIP   10.98.106.37   172.28.242.11   5000/TCP   137m


---
ingress controller

https://kubernetes.github.io/ingress-nginx/deploy/

In [ ]:
helm upgrade --install ingress-nginx ingress-nginx --repo https://kubernetes.github.io/ingress-nginx \
--namespace ingress-nginx --create-namespace

---
ingress controller

In [ ]:
kubectl apply -f https://raw.githubusercontent.com/kubernetes/ingress-nginx/controller-v1.10.0/deploy/static/provider/cloud/deploy.yaml

---
ingress

In [1]:
cat << EOF > docker-registry-ingress.yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: docker-registry
  namespace: docker-registry
spec:
  defaultBackend:
    service:
      name: docker-registry
      port:
        number: 5000
EOF

---
ingress tls secret

In [ ]:
sudo systemctl stop traefik
sudo certbot certonly --standalone --agree-tos --email saintway.sakaru@gmail.com -d registry.dtype.info -d jhub.dtype.info --renew-by-default

```
Saving debug log to /var/log/letsencrypt/letsencrypt.log
Plugins selected: Authenticator standalone, Installer None
Renewing an existing certificate for registry.dtype.info and jhub.dtype.info

IMPORTANT NOTES:
 - Congratulations! Your certificate and chain have been saved at:
   /etc/letsencrypt/live/registry.dtype.info/fullchain.pem
   Your key file has been saved at:
   /etc/letsencrypt/live/registry.dtype.info/privkey.pem
   Your certificate will expire on 2024-07-15. To obtain a new or
   tweaked version of this certificate in the future, simply run
   certbot again. To non-interactively renew *all* of your
   certificates, run "certbot renew"
 - If you like Certbot, please consider supporting our work by:

   Donating to ISRG / Let's Encrypt:   https://letsencrypt.org/donate
   Donating to EFF:                    https://eff.org/donate-le
```

In [ ]:
sudo systemctl start traefik
sudo tljh-config reload hub

In [8]:
kubectl apply -f - << EOF
apiVersion: v1
kind: Secret
metadata:
  name: registry-tls
  namespace: docker-registry
data:
  tls.crt: $(sudo base64 -w 0 /etc/letsencrypt/live/registry.dtype.info/fullchain.pem)
  tls.key: $(sudo base64 -w 0 /etc/letsencrypt/live/registry.dtype.info/privkey.pem)
type: kubernetes.io/tls
EOF

secret/registry-tls created


---
ingress

In [1]:
cat << EOF > docker-registry-ingress.yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    kubernetes.io/tls-acme: 'true'
  name: docker-registry
  namespace: docker-registry
spec:
  tls:
  - hosts:
    - registry.dtype.info
    secretName: registry-tls
  rules:
  - host: registry.dtype.info
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: docker-registry
            port:
              number: 5000
EOF

In [2]:
kubectl apply -f docker-registry-ingress.yaml

ingress.networking.k8s.io/docker-registry configured


In [ ]:
kubectl get pods -A

NAMESPACE              NAME                                                   READY   STATUS    RESTARTS   AGE
docker-registry        docker-registry-68f8d8dc67-qkp57                       1/1     Running   0          25m
kube-flannel           kube-flannel-ds-lcndk                                  1/1     Running   0          7h47m
kube-flannel           kube-flannel-ds-lpr4m                                  1/1     Running   0          7h52m
kube-system            coredns-76f75df574-7sbbl                               1/1     Running   0          7h42m
kube-system            coredns-76f75df574-zmsv2                               1/1     Running   0          7h42m
kube-system            etcd-free-vpn                                          1/1     Running   1          8h
kube-system            kube-apiserver-free-vpn                                1/1     Running   1          8h
kube-system            kube-controller-manager-free-vpn                       1/1     Running   0         

In [1]:
kubectl get ingress -A

NAMESPACE         NAME              CLASS    HOSTS                 ADDRESS           PORTS     AGE
docker-registry   docker-registry   <none>   registry.dtype.info   jhub.dtype.info   80, 443   29h


In [2]:
curl https://registry.dtype.info

---
https://cert-manager.io/docs/tutorials/acme/nginx-ingress/

In [ ]:
kubectl apply -f https://github.com/cert-manager/cert-manager/releases/download/v1.14.4/cert-manager.yaml

In [ ]:
cat << EOF > letsencrypt-staging.yaml
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: letsencrypt-staging
spec:
  acme:
    # The ACME server URL
    server: https://acme-staging-v02.api.letsencrypt.org/directory
    # Email address used for ACME registration
    email: saintway.sakaru@gmail.com
    # Name of a secret used to store the ACME account private key
    privateKeySecretRef:
      name: letsencrypt-staging
    # Enable the HTTP-01 challenge provider
    solvers:
    - http01:
        ingress:
            ingressClassName: nginx
EOF

In [ ]:
kubectl apply -f letsencrypt-staging.yaml

In [ ]:
kubectl apply -f - << EOF
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: letsencrypt-prod
spec:
  acme:
    # The ACME server URL
    server: https://acme-v02.api.letsencrypt.org/directory
    # Email address used for ACME registration
    email: saintway.sakaru@gmail.com
    # Name of a secret used to store the ACME account private key
    privateKeySecretRef:
      name: letsencrypt-prod
    # Enable the HTTP-01 challenge provider
    solvers:
      - http01:
          ingress:
            ingressClassName: nginx
EOF

---
https://cert-manager.io/docs/configuration/acme/

In [1]:
PROJECT_ID=sandbox-sheng
gcloud iam service-accounts create dns01-solver --display-name 'dns01-solver'

Created service account [dns01-solver].


In [2]:
gcloud projects add-iam-policy-binding $PROJECT_ID \
   --member serviceAccount:dns01-solver@$PROJECT_ID.iam.gserviceaccount.com \
   --role roles/dns.admin

Updated IAM policy for project [sandbox-sheng].
bindings:
- members:
  - serviceAccount:agi-chat@sandbox-sheng.iam.gserviceaccount.com
  - serviceAccount:service-139287821772@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-139287821772@container-engine-robot.iam.gserviceaccount.com
  role: roles/container.serviceAgent
- members:
  - serviceAccount:service-139287821772@dataflow-service-producer-prod.iam.gserviceaccount.com
  role: roles/dataflow.serviceAgent
- members:
  - serviceAccount:dns01-solver@sandbox-sheng.iam.gserviceaccount.com
  role: roles/dns.admin
- members:
  - serviceAccount:139287821772-compute@developer.gserviceaccount.com
  - serviceAccount:139287821772@cloudservices.gserviceaccount.com
  - serviceAccount:service-139287821772@containerregistry.iam.gserviceaccount.com
  role: roles/editor
- members:
  - user:andrew.saintway@gmail.com
  - user:saintway@jhub.name
  role: roles/owner
etag: BwYWK48ruTk=
versio

In [3]:
gcloud iam service-accounts keys create key.json \
   --iam-account dns01-solver@$PROJECT_ID.iam.gserviceaccount.com

created key [1cf8a588be1f2eb35acabab6384fb78b475990b9] of type [json] as [key.json] for [dns01-solver@sandbox-sheng.iam.gserviceaccount.com]


In [4]:
kubectl create secret generic clouddns-dns01-solver-svc-acct \
   --from-file=key.json

secret/clouddns-dns01-solver-svc-acct created


In [ ]:
kubectl apply -f - << EOF
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: letsencrypt-prod
spec:
  acme:
    server: https://acme-v02.api.letsencrypt.org/directory
    email: saintway.sakaru@gmail.com
    privateKeySecretRef:
      name: letsencrypt-prod
    solvers:
    - dns01:
        cloudDNS:
          # The ID of the GCP project
          project: $PROJECT_ID
          # This is the secret used to access the service account
          serviceAccountSecretRef:
            name: clouddns-dns01-solver-svc-acct
            key: key.json
EOF